<a href="https://colab.research.google.com/github/colinmcnamara/Digital_Deming/blob/main/Digital_Deming.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Set up your local environment 

On intial run of "import colab_env" it will create vars.env in the root of your Google Drive

COLAB_ENV = Active

export OPENAI_API_KEY = "your_key"

export SERPAPI_API_KEY = "your_key"


Install my local packages

In [155]:
pip install OpenAI langchain pydantic faiss-gpu colab-env unstructured chromadb tiktoken pdf2image faiss-gpu

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Yo, let's import our local env so we don't have to show the world our keys

In [156]:
import colab_env

Let's import our python modules

In [157]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.text_splitter import CharacterTextSplitter
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain import OpenAI, VectorDBQA
from langchain.chains import RetrievalQA
from langchain.chains import RetrievalQAWithSourcesChain
from langchain.document_loaders import DirectoryLoader
from langchain.prompts import PromptTemplate
from langchain.llms import OpenAI
from langchain.agents import load_tools
from langchain.agents import initialize_agent
from langchain.agents import AgentType
from langchain.chat_models import ChatOpenAI
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage
)
import magic
import os
import nltk
import textwrap



Let's set the temperature of our AI models

In [158]:
llm = OpenAI(temperature=0)
chat = ChatOpenAI(temperature=0)

Let's specify the directory and file type for our Demming source docs

In [159]:
loader = DirectoryLoader('/content/gdrive/MyDrive/Projects/Digital_Deming/src_content', glob='**/*.pdf')

Load our documents


In [160]:
documents = loader.load()

Check how many documents are loaded

In [161]:
len (documents)

1

Let's split our document up into chunks

In [162]:
#text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
text_splitter = RecursiveCharacterTextSplitter(
    # Set a really small chunk size, just to show.
    chunk_size = 1000,
    chunk_overlap  = 20,
    length_function = len,
    separators=["\n\n", "\n", " ", ""]
)

Let' create a variable named texts, and split these documents into it

In [163]:
texts = text_splitter.split_documents(documents)

Let's tell it that we want it to use OpenAI embeddings, and tell it what env variable the key is held

In [164]:
embeddings = OpenAIEmbeddings(openai_api_key=os.environ['OPENAI_API_KEY'])

Let's create a variable named docsearch, and pull from Chroma vectorstore including texts, and the embeddings)

In [165]:
docsearch = Chroma.from_documents(texts, embeddings)

define qa chain, which pulls from VectorDBQA, and use the LLM chain named OpenAI. We specify that it "Stuff"'s all the data in. We could use "map_reduce" instead to optimize for performance and scale. Let's tell it to use the vectorstore we defined earlier named docsearch

note, i'm trying RetrievalQA because VectorDBQA is depricated

In [166]:
qa = VectorDBQA.from_chain_type(llm=OpenAI(), chain_type="stuff", vectorstore=docsearch, return_source_documents=True)
query = "Describe in depth, what is the foundation of quality. Provide three examples using bullet points"
result = qa({"query": query})

In [167]:
result['result']

"\n\nThe foundation of quality is a set of principles, processes, and methods that ensure that products and services meet or exceed customer expectations. \n\nExamples: \n• Quality Control: Quality control involves the use of tools, processes, and techniques to ensure that products and services meet the customer's expectations. \n• Quality Assurance: Quality assurance is the process of verifying that products and services meet the customer's expectations. It involves the use of techniques such as inspections, audits, and analysis to identify and address problems. \n• Quality Management: Quality management is the process of managing the quality of products and services in order to meet customer requirements. It involves the use of tools such as planning, training, and problem solving to improve the quality of products and services."

In [168]:
result['source_documents']

[Document(page_content='1) AGREE: Advisory Group on Reliability of Electronic Equipment 2) AOTS: Association for Overseas Technical Scholarship 3) APO: Asian Productivity Organization 4) ASA: American Standards Association 5) ASQC: American Society for Quality Control 6) BS: British Standards 7) CCS: Civil Communication Section 8) COSCO: Control System Committee 9) EOQC: European Organization for Quality Control 10) EOQ: European Organization for Quality 11) EFQM: European Foundation for Quality Management 12) FMES: Japan Federation of Managerial Engineering Societies\n\n524\n\nA Parallel Chronology of Prof. Kaoru Ishikawa’s Life and TQC in Japan', metadata={'source': '/content/gdrive/MyDrive/Projects/Digital_Deming/src_content/ParallelChronology_Ver2_150717.pdf'}),
 Document(page_content='Major Events in the Development of QC\n\nCourse. (July–Dec. 1957, JUSE)\n\n\uf09f The Chugoku Quality Control Research Group was established. (JSA Hiroshima) \uf09f The Design of Experiment Seminar w

In [169]:
qa = VectorDBQA.from_chain_type(llm=OpenAI(), chain_type="stuff", vectorstore=docsearch, return_source_documents=True)
#query = "what is the impact of JUSE on TQM?"
#query = "what is the purpouse of life?"
#query = "what does demming like for lunch"
#query = "is quality like a human"
#query = "what is the fourth rule of TQM"
#query = "what is the most important concept in TQM"
#query =  "WHo won the deming prize?"
#query = "Is TQM like lean?"
#query = "How are TQM and Lean different?"
query = "What is lean?"

result = qa({"query": query})

In [170]:
result['result']

' Lean is a method for improving the efficiency of production processes by eliminating waste and increasing value.'

In [171]:
result['source_documents']

[Document(page_content='held. (July, JUSE)\n\n\uf09f Governmental Building Construction Q-S Round-table conference was held. (JSA)\n\n\uf09f The Mexican Quality Control Training\n\n(Age 62)\n\n515\n\nA Parallel Chronology of Prof. Kaoru Ishikawa’s Life and TQC in Japan\n\nYear\n\nProf. Ishikawa’s Life\n\nMajor Events in the Development of QC\n\n\uf09f He became a researcher at the\n\nTeam visited Japan. (JICA/JUSE)\n\nComponent Measurement Department, Laboratory of Metrology, the Agency of Industrial Science and Technology. \uf09f He became the chair of the executive\n\ncommittee of the 20th National Meeting of Standardization. (JSA)\n\n\uf09f He became the chair of the organization committee of the Mexican Quality Control Training Course. (JICA/JUSE)\n\n\uf09f He was awarded with the Medal with\n\nBlue Ribbon for his contribution to the promotion of industrial standardization. \uf09f He became an honorary member of the Argentine Association for Quality Control.\n\n\uf09f The 1st PL Wo

# The History and Evolution of Total Quality Management (TQM): The Contributions of Kaoru Ishikawa and the Japanese Union of Scientists and Engineers (JUSE)

## I. Introduction
- Definition of Total Quality Management (TQM)
- Brief introduction of the significance of TQM in the modern business world
- Introduction to Kaoru Ishikawa and the Japanese Union of Scientists and Engineers (JUSE)
- Thesis Statement

In [172]:
qa = VectorDBQA.from_chain_type(llm=OpenAI(), chain_type="stuff", vectorstore=docsearch, return_source_documents=True)
#query = "what is the impact of JUSE on TQM?"
#query = "what is the purpouse of life?"
#query = "what does demming like for lunch"
#query = "is quality like a human"
#query = "what is the fourth rule of TQM"
query = "what is Total Quality Management (TQM)"
result = qa({"query": query})

In [173]:
result['result']

" Total Quality Management (TQM) is an approach to business management that seeks to maximize customer satisfaction by improving quality in all areas of an organization's operations. It is a process that involves integrating all functions of an organization to achieve continuous improvement of quality. It is used to achieve a competitive advantage through improved customer satisfaction and increased efficiency."

In [174]:
result['source_documents']

[Document(page_content='1) AGREE: Advisory Group on Reliability of Electronic Equipment 2) AOTS: Association for Overseas Technical Scholarship 3) APO: Asian Productivity Organization 4) ASA: American Standards Association 5) ASQC: American Society for Quality Control 6) BS: British Standards 7) CCS: Civil Communication Section 8) COSCO: Control System Committee 9) EOQC: European Organization for Quality Control 10) EOQ: European Organization for Quality 11) EFQM: European Foundation for Quality Management 12) FMES: Japan Federation of Managerial Engineering Societies\n\n524\n\nA Parallel Chronology of Prof. Kaoru Ishikawa’s Life and TQC in Japan', metadata={'source': '/content/gdrive/MyDrive/Projects/Digital_Deming/src_content/ParallelChronology_Ver2_150717.pdf'}),
 Document(page_content='Medal. (ASQC)\n\n\uf09f The Pharmaceutical Manufacturing and Quality Control (GMP) Seminar was held. (Sep., JUSE)\n\n\uf09f The Japan Prefabricated Construction\n\nSuppliers and Manufacturers Associa

In [175]:
# Create a list to stuff section results into 
introduction = []

#append result  to the array
introduction.append(result)

#print the array
print(introduction)

[{'query': 'what is Total Quality Management (TQM)', 'result': " Total Quality Management (TQM) is an approach to business management that seeks to maximize customer satisfaction by improving quality in all areas of an organization's operations. It is a process that involves integrating all functions of an organization to achieve continuous improvement of quality. It is used to achieve a competitive advantage through improved customer satisfaction and increased efficiency.", 'source_documents': [Document(page_content='1) AGREE: Advisory Group on Reliability of Electronic Equipment 2) AOTS: Association for Overseas Technical Scholarship 3) APO: Asian Productivity Organization 4) ASA: American Standards Association 5) ASQC: American Society for Quality Control 6) BS: British Standards 7) CCS: Civil Communication Section 8) COSCO: Control System Committee 9) EOQC: European Organization for Quality Control 10) EOQ: European Organization for Quality 11) EFQM: European Foundation for Quality

In [176]:
qa = VectorDBQA.from_chain_type(llm=OpenAI(), chain_type="stuff", vectorstore=docsearch, return_source_documents=True)
#query = "what is the impact of JUSE on TQM?"
#query = "what is the purpouse of life?"
#query = "what does demming like for lunch"
#query = "is quality like a human"
#query = "what is the fourth rule of TQM"
query = "Write a detailed introduction of the significance of TQM in the modern business world"
result = qa({"query": query})

In [177]:
result['result']

' Total Quality Management (TQM) is an approach to business management that focuses on improving the quality of products and services in order to increase customer satisfaction. TQM has become increasingly important in the modern business world as organizations strive to remain competitive. By implementing TQM, organizations can reduce costs, improve customer service, increase efficiency, and develop long-term customer relationships. TQM also encourages organizations to embrace continuous improvement, strive for excellence, and focus on customer needs. As a result, TQM can help organizations improve customer loyalty, increase market share, and achieve long-term success.'

In [178]:
#append result  to the array
introduction.append(result)

#print the array
print(introduction)

[{'query': 'what is Total Quality Management (TQM)', 'result': " Total Quality Management (TQM) is an approach to business management that seeks to maximize customer satisfaction by improving quality in all areas of an organization's operations. It is a process that involves integrating all functions of an organization to achieve continuous improvement of quality. It is used to achieve a competitive advantage through improved customer satisfaction and increased efficiency.", 'source_documents': [Document(page_content='1) AGREE: Advisory Group on Reliability of Electronic Equipment 2) AOTS: Association for Overseas Technical Scholarship 3) APO: Asian Productivity Organization 4) ASA: American Standards Association 5) ASQC: American Society for Quality Control 6) BS: British Standards 7) CCS: Civil Communication Section 8) COSCO: Control System Committee 9) EOQC: European Organization for Quality Control 10) EOQ: European Organization for Quality 11) EFQM: European Foundation for Quality

In [179]:
qa = VectorDBQA.from_chain_type(llm=OpenAI(), chain_type="stuff", vectorstore=docsearch, return_source_documents=True)
#query = "what is the impact of JUSE on TQM?"
#query = "what is the purpouse of life?"
#query = "what does demming like for lunch"
#query = "is quality like a human"
#query = "what is the fourth rule of TQM"
#query = "Write a detailed introduction of the significance of TQM in the modern business world"
query = "Introduction to Kaoru Ishikawa and the Japanese Union of Scientists and Engineers (JUSE)"
result = qa({"query": query})

In [180]:
result['result']

' In 1962, Kaoru Ishikawa belonged to the 4th Fuel Engineering Laboratory (Applied Combustible Science) at the University of Tokyo. He became a lecturer of the Introductory Course for Design of Experiment, a member of the ISO/TC 102 Japan Committee, and the chair of the Sampling Subcommittee. He also became a lecturer of the QC Seminar for Managers, a member of the Council for Industrial Rationalization, the chair of the Editorial Committee of the JUSE English Report, and the chair of the Organizing Committee of the JUSE Course for Purchasing. He was involved in other activities such as the Inspection Seminar, the QC Correspondence Basic Course for Foremen, the Japanese Society for Quality Control Internationally-operating Company QC Research Delegation, and the “In-Plant Group Training Program in the Field of Quality Improvement for Industrial Products.”'

In [181]:
#append result  to the array
introduction.append(result)

#print the array
print(introduction)

[{'query': 'what is Total Quality Management (TQM)', 'result': " Total Quality Management (TQM) is an approach to business management that seeks to maximize customer satisfaction by improving quality in all areas of an organization's operations. It is a process that involves integrating all functions of an organization to achieve continuous improvement of quality. It is used to achieve a competitive advantage through improved customer satisfaction and increased efficiency.", 'source_documents': [Document(page_content='1) AGREE: Advisory Group on Reliability of Electronic Equipment 2) AOTS: Association for Overseas Technical Scholarship 3) APO: Asian Productivity Organization 4) ASA: American Standards Association 5) ASQC: American Society for Quality Control 6) BS: British Standards 7) CCS: Civil Communication Section 8) COSCO: Control System Committee 9) EOQC: European Organization for Quality Control 10) EOQ: European Organization for Quality 11) EFQM: European Foundation for Quality

In [182]:
qa = VectorDBQA.from_chain_type(llm=OpenAI(), chain_type="stuff", vectorstore=docsearch, return_source_documents=True)
#query = "what is the impact of JUSE on TQM?"
#query = "what is the purpouse of life?"
#query = "what does demming like for lunch"
#query = "is quality like a human"
#query = "what is the fourth rule of TQM"
#query = "Write a detailed introduction of the significance of TQM in the modern business world"
#query = "Introduction to Kaoru Ishikawa and the Japanese Union of Scientists and Engineers (JUSE)"
query = " Write a Thesis Statement regarding TQM, Kaoru Ishikawa, Japanese Union of Scientists and engineers."
result = qa({"query": query})

In [183]:
result['result']

"  Thesis statement: Kaoru Ishikawa's life and work with the Japanese Union of Scientists and Engineers helped to promote Total Quality Management in Japan."

In [184]:
#append result  to the array
introduction.append(result)

#print the array
print(introduction)

[{'query': 'what is Total Quality Management (TQM)', 'result': " Total Quality Management (TQM) is an approach to business management that seeks to maximize customer satisfaction by improving quality in all areas of an organization's operations. It is a process that involves integrating all functions of an organization to achieve continuous improvement of quality. It is used to achieve a competitive advantage through improved customer satisfaction and increased efficiency.", 'source_documents': [Document(page_content='1) AGREE: Advisory Group on Reliability of Electronic Equipment 2) AOTS: Association for Overseas Technical Scholarship 3) APO: Asian Productivity Organization 4) ASA: American Standards Association 5) ASQC: American Society for Quality Control 6) BS: British Standards 7) CCS: Civil Communication Section 8) COSCO: Control System Committee 9) EOQC: European Organization for Quality Control 10) EOQ: European Organization for Quality 11) EFQM: European Foundation for Quality

Let's see if we can output this to markdown

In [185]:
concatenated_data = [str(item) for item in introduction]
for entry in concatenated_data:
    print("Processing entry: ")
    messages = [
        SystemMessage(content="You are an AI that takes dictionary data and translates it into well formatted markdown. You create sections and subsections to communicate as much information as possible"),
        HumanMessage(content=entry)
    ]
    result = chat(messages)

    print("Result: ", result)

Processing entry: 
Result:  content="# Total Quality Management (TQM)\n\nTotal Quality Management (TQM) is an approach to business management that seeks to maximize customer satisfaction by improving quality in all areas of an organization's operations. It is a process that involves integrating all functions of an organization to achieve continuous improvement of quality. It is used to achieve a competitive advantage through improved customer satisfaction and increased efficiency.\n\n## Source Documents\n\n- AGREE: Advisory Group on Reliability of Electronic Equipment\n- AOTS: Association for Overseas Technical Scholarship\n- APO: Asian Productivity Organization\n- ASA: American Standards Association\n- ASQC: American Society for Quality Control\n- BS: British Standards\n- CCS: Civil Communication Section\n- COSCO: Control System Committee\n- EOQC: European Organization for Quality Control\n- EOQ: European Organization for Quality\n- EFQM: European Foundation for Quality Management\n- 

In [186]:

  messages = [
      SystemMessage(content="You are an assistant helping me research a paper"),
      HumanMessage(content="return an outline for the book - The History and Evolution of Total Quality Management (TQM): The Contributions of Kaoru Ishikawa and the Japanese Union of Scientists and Engineers (JUSE) ")
  ]
  result = chat(messages)

  print("Result: ", result)

Result:  content="I. Introduction\n- Brief overview of Total Quality Management (TQM)\n- Importance of understanding the history and evolution of TQM\n- Thesis statement\n\nII. Early Developments in TQM\n- Origins of TQM in the United States\n- Early criticisms and limitations of TQM\n- Introduction of TQM to Japan\n\nIII. Kaoru Ishikawa and the Japanese Union of Scientists and Engineers (JUSE)\n- Background and contributions of Kaoru Ishikawa\n- Formation and purpose of JUSE\n- Ishikawa's development of the Ishikawa diagram and other quality tools\n\nIV. Spread of TQM in Japan\n- Adoption of TQM by Japanese companies\n- Impact of TQM on Japanese industry and economy\n- Comparison of Japanese and American approaches to TQM\n\nV. Globalization of TQM\n- Spread of TQM to other countries\n- Challenges and criticisms of TQM in different cultural contexts\n- Evolution of TQM beyond its original principles\n\nVI. Conclusion\n- Summary of key points\n- Implications of TQM for contemporary bus

In [187]:
#from langchain.prompts import PromptTemplate
prompt_template = """Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.
Provide responses in Markdown

Here is the structure of the paper:

The History and Evolution of Total Quality Management (TQM): The Contributions of Kaoru Ishikawa and the Japanese Union of Scientists and Engineers (JUSE)
I. Introduction
Definition of Total Quality Management (TQM)
Brief introduction of the significance of TQM in the modern business world
Introduction to Kaoru Ishikawa and the Japanese Union of Scientists and Engineers (JUSE)
Thesis Statement
III. The Birth and Evolution of TQM
Introduction to the concept of TQM
Overview of the principles and practices of TQM
The role of TQM in post-World War II Japan's industrial comeback
Adoption and evolution of TQM in the United States and around the world
IV. The Role of Kaoru Ishikawa in the Development of TQM
Introduction to Ishikawa: his background and career
Ishikawa's contribution to the development of Quality Circles
The development and importance of the Ishikawa (cause-and-effect) Diagram
Ishikawa's philosophical contribution to TQM: the 'next process is your customer'
V. The Role of the Japanese Union of Scientists and Engineers (JUSE)
Background of JUSE and its role in the development of quality management in Japan
Introduction of the Deming Prize by JUSE and its significance
JUSE's role in promoting TQM and its principles in Japan and globally
VI. The Impact of TQM, Ishikawa, and JUSE on Contemporary Business Practices
Analysis of the role TQM plays in today's business and management practices
The legacy of Ishikawa's methods and tools in current quality management
Influence of JUSE's principles in contemporary quality initiatives and awards
VII. Conclusion
Summary of the importance and influence of TQM, Ishikawa, and JUSE in the field of quality management
Restatement of Thesis
Future implications of the principles and practices of TQM, Ishikawa, and JUSE in the global business landscape

{context}

Question: {question}
Answer as if you are a quality management consultant. Use specific dates and references. Use bullet points and tabels where appropriate. Reference source documents:

"""
PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)
chain_type_kwargs = {"prompt": PROMPT}
qa = RetrievalQA.from_chain_type(llm=OpenAI(), chain_type="stuff", retriever=docsearch.as_retriever(), chain_type_kwargs=chain_type_kwargs)
query = "Who won the Deming prize?"
qa.run(query)

'The first winners of the Deming Prize were Showa Denko, Tanabe Pharmaceuticals, Fuji Iron & Steel, and Yawata Iron & Steel in September 1980. The first construction company to win the prize was Mitsubishi Heavy Industries’ Kobe Shipyard in November 1986.'

In [188]:
query = "when was the last deming prize awarded"
qa.run(query)

'The last Deming Prize was awarded in 2019. The award ceremony took place in Tokyo, Japan on October 29, 2019. \n\nThe 2019 Deming Prize winners were: \n• Individuals: Dr. Toshio Nakanishi (Recipient of the Grand Prize) and Dr. Takeshi Yamada (Recipient of the Prize of Honor) \n• Companies: Toyota Boshoku Corporation, Daiichi Sankyo Company, Ltd., and Ricoh Company, Ltd. \n\nThe Deming Prize recognizes individuals and organizations who have achieved outstanding results in terms of quality control and management. It is awarded by the Japanese Union of Scientists and Engineers (JUSE). The Deming Prize was established in 1951 at the same time as the first Quality Control (QC) Conference. \n\nReferences: \n• The 2019 Deming Prize Award Ceremony, https://www.juse.or.jp/en/deming/awards/2019/ \n• Chronology of Prof. Kaoru Ishikawa’s Life and TQC in Japan, https://www.juse.or.jp/en/deming/history/chronology/index.html'

In [189]:
query = "who invented QC circle?"
qa.run(query)

"Kaoru Ishikawa is widely credited with inventing Quality Circles. He is credited with popularizing the concept in Japan in the 1960s, and the concept has since spread around the world. \n\nIn the early 1960s, Ishikawa developed the concept of Quality Circles in order to improve the quality of production processes and reduce costs. He also developed the Ishikawa Diagram, a tool used to graphically identify potential sources of variation in a product or process. \n\nIn 1971, Ishikawa became the President of the Japanese Society for Quality Control and established Quality Circles as a formalized process for improving quality and efficiency. He also developed the concept of Total Quality Control (TQC), which focused on continual improvement and customer satisfaction. \n\nIshikawa's work was instrumental in the development of the international quality management movement. His methods and tools are still widely used in the field of quality management today."

In [190]:
query = "when was the QC Circle Seminar for Top Management held"
qa.run(query)

'\nThe QC Circle Seminar for Top Management was held in 1973. This was organized by Professor Kaoru Ishikawa and the Japanese Union of Scientists and Engineers (JUSE). The event was held in April and was hosted by the QC Circle Headquarters.'

# The History and Evolution of Total Quality Management (TQM): The Contributions of Kaoru Ishikawa and the Japanese Union of Scientists and Engineers (JUSE)

## I. Introduction
* Definition of Total Quality Management (TQM)
* Brief introduction of the significance of TQM in the modern business world
* Introduction to Kaoru Ishikawa and the Japanese Union of Scientists and Engineers (JUSE)
* Thesis Statement

In [191]:
query = "write a Definition of Total Quality Management (TQM)"
qa.run(query)

'Total Quality Management (TQM) is a holistic approach to quality management that was first introduced in Japan in the 1950s. It is based on the principles of continuous improvement, customer focus, and process management. TQM seeks to improve the quality of products and services by focusing on process efficiency, customer satisfaction, and employee involvement.\n\nThe concept of TQM was developed by Kaoru Ishikawa of the Japanese Union of Scientists and Engineers (JUSE) in 1950. The core principles of TQM include: \n\n• Continuous improvement: A systematic approach to improving processes and services through the use of data analysis, feedback, and employee involvement.\n\n• Customer focus: An emphasis on understanding customer needs and expectations in order to provide products and services that meet or exceed those expectations.\n\n• Process management: A method of managing processes and services by analyzing them in order to identify areas for improvement and developing plans for im

In [192]:
query = "write a Brief introduction of the significance of TQM in the modern business world"
qa.run(query)

"Total Quality Management (TQM) is a business philosophy and practice that has been adopted by a wide range of industries over the past several decades. TQM focuses on the elimination of waste and inefficiency in all aspects of an organization's operations in order to improve customer satisfaction. TQM has been shown to reduce costs, improve efficiency, and promote employee satisfaction and engagement.\n\nSpecifically, TQM is a comprehensive system for managing and improving all aspects of an organization's performance, including quality, productivity, safety, customer satisfaction, and environmental impacts. It is a process-oriented approach that involves the continuous monitoring and improvement of processes, products, and services. TQM is rooted in the principles of continuous improvement and customer satisfaction, and is often referred to as Total Quality Control (TQC).\n\nThe principles of TQM have been embraced by organizations around the world, including the American Society for

In [193]:
query = "write an Introduction to Kaoru Ishikawa and the Japanese Union of Scientists and Engineers (JUSE)"
qa.run(query)

'\nKaoru Ishikawa was a Japanese professor, author, and quality management consultant. He was born on July 13th, 1915 in Tokyo, Japan. He is best known for his work in the field of Total Quality Management (TQM) and his contribution to the development of the Ishikawa (cause-and-effect) Diagram, also known as a Fishbone Diagram.\n\nIn 1964, Ishikawa joined the Japanese Union of Scientists and Engineers (JUSE) and became a part of the Quality Control (QC) movement in Japan. As a professor at the University of Tokyo, he developed the QC Seminar for Managers and the QC Seminar for Supervisors. He also wrote multiple books on the subject of QC and published the "Supervisor Special Issue" of Hinshitsu Kanri (Statistical Quality Control). \n\nIn addition, Ishikawa was a major contributor to the development and implementation of the ISO/TC 102 (iron ore). He was also appointed as the chair of the Sampling Subcommittee of the Japan Committee and later became the chair of the Japan Consumers’ As

In [194]:
query = "write an Thesis statement "
qa.run(query)

'Thesis statement: Total Quality Management (TQM) has been a powerful force in the business world since its introduction in the 1940s, with the efforts of Kaoru Ishikawa and the Japanese Union of Scientists and Engineers (JUSE) playing an integral role in its development. This paper will explore the history and evolution of TQM, the contributions of Ishikawa and JUSE, the impact of their work on contemporary business practices, and the implications of TQM for the future.'

## II. Early History of Quality Management
- Brief description of quality management before TQM
- Introduction to foundational concepts: Inspection, Quality Control, and Quality Assurance
- The rise of statistical process control and its significance

In [195]:
query = "write  Early History of Quality Management Brief description of quality management before TQM"
qa.run(query)

'Early History of Quality Management\n\nBefore Total Quality Management (TQM), the quality management approach was based on the following principles and practices:\n\n• 1931: Walter Shewhart’s Economic Control of Quality of Manufactured Products was published. This laid the foundation for statistical process control (SPC).\n• 1931: Yasushi Ishida devised the control chart, which he called “makimono,” and used to control the production process of lightbulbs. \n• 1931: Kanzo Kiribuchi introduced quality control as “Hinshitsu Tosei” in his book The Science of Industrial Control. \n• 1935: Pearson’s “The Application of Statistical Methods to Industrial Standardization and Quality Control,” which later became BS 600, was published. \n• 1936: Shewhart’s Statistical Method from the Viewpoint of Quality Control was published. \n• 1949: The first Deming Prize was awarded to Nippon Telephone and Telegraph, recognizing its achievement in quality control. \n• 1954: The Japanese Union of Scientists

In [196]:
query = "write Introduction to foundational concepts: Inspection, Quality Control, and Quality Assurance"
qa.run(query)

'\nIntroduction to foundational concepts: Inspection, Quality Control, and Quality Assurance\n\nInspection: \n• Inspection is the process of objectively examining a product to ensure that it meets predetermined specifications and requirements.\n• After World War II, Japan began to adopt inspection as part of its industrialization efforts. \n• In 1949, the Quality Control Research Group was established which marked the beginnings of Quality Control in Japan (JUSE).\n\nQuality Control: \n• Quality Control (QC) is an approach to quality management that focuses on conforming to predetermined standards and requirements. \n• QC was adopted by several Japanese companies in 1949 under the guidance of Sarasohn.\n• In 1950, the Research Institute for Electrical Communication introduced sampling inspection on purchased products (Nippon Telegraph and Telephone Public Corporation). \n• In 1951, consultations on QC activities started at several companies by Eizaburo Nishibori, etc. (May, JMA) \n• In

In [197]:
query = "The rise of statistical process control and its significance"
qa.run(query)

'\nThe rise of statistical process control and its significance can be traced back to 1931 with the publication of Shewhart’s Economic Control of Quality of Manufactured Products. This was followed by Yasushi Ishida’s control chart, which he called “makimono,” and used to control the production process of lightbulbs. Kanzo Kiribuchi introduced quality control as “Hinshitsu Tosei” in his book The Science of Industrial Control. Pearson’s “The Application of Statistical Methods to Industrial Standardization and Quality Control,” which later became BS 600, was published in 1935.\n\nIn 1936, Shewhart’s Statistical Method from the Viewpoint of Quality Control was published. This laid the groundwork for the development of statistical process control (SPC). SPC is an effective tool for monitoring and controlling processes. It is based on the principles of using statistical methods to analyze data and identify trends and patterns in production processes.\n\nThe following table summarizes the ri

## III. The Birth and Evolution of TQM
- Introduction to the concept of TQM
- Overview of the principles and practices of TQM
- The role of TQM in post-World War II Japan's industrial comeback
- Adoption and evolution of TQM in the United States and around the world

In [198]:
query = "The rise of statistical process control and its significance"
qa.run(query)

'The rise of statistical process control (SPC) and its significance has been a major force in the development of quality management. It began in the early 1900s with the publication of Walter Shewhart\'s "Economic Control of Quality of Manufactured Products" in 1931 and Yasushi Ishida\'s development of the control chart, which he called "makimono," in the same year. \n\nKanzo Kiribuchi introduced quality control as "Hinshitsu Tosei" in his 1931 book "The Science of Industrial Control." In 1935, Pearson published "The Application of Statistical Methods to Industrial Standardization and Quality Control," which later became BS 600. Shewhart\'s "Statistical Method from the Viewpoint of Quality Control" was published in 1936. \n\nThis laid the groundwork for Kaoru Ishikawa\'s contributions to the development of Total Quality Management (TQM). In 1947, he published "Control Chart," which was the first book to present the Seven Basic Tools of Quality Control. These tools, which include contro

In [199]:
query = "The rise of statistical process control and its significance"
qa.run(query)

'As a quality management consultant, I can tell you that statistical process control (SPC) has become increasingly important in modern business and management practices. The rise of SPC began in 1931, when Walter A. Shewhart published his famous book Economic Control of Quality of Manufactured Products. This book introduced the concept of SPC to the world. \n\nIn the same year, Yasushi Ishida devised the control chart, which he called “makimono,” and used it to control the production process of lightbulbs. Also in 1931, Kanzo Kiribuchi introduced quality control as “Hinshitsu Tosei” in his book The Science of Industrial Control. \n\nIn 1935, Pearson’s “The Application of Statistical Methods to Industrial Standardization and Quality Control,” which later became BS 600, was published. This book further reinforced the importance of SPC in the modern business world. \n\nIn 1936, Shewhart’s Statistical Method from the Viewpoint of Quality Control was published. This book provided a comprehe

In [200]:
query = "The rise of statistical process control and its significance"
qa.run(query)

'The rise of statistical process control (SPC) and its significance can be seen as a major milestone in the development of Total Quality Management (TQM). The earliest known examples of SPC can be traced back to the early 1930s, with several key publications and events that took place in this period playing a major role in its development. \n\nThese include: \n\n- 1931: Shewhart’s Economic Control of Quality of Manufactured Products was published. \n- 1931: Yasushi Ishida devised the control chart, which he called “makimono,” and used to control the production process of lightbulbs. \n- 1931: Kanzo Kiribuchi introduced quality control as “Hinshitsu Tosei” in his book The Science of Industrial Control. \n- 1935: Pearson’s “The Application of Statistical Methods to Industrial Standardization and Quality Control,” which later became BS 600, was published. \n- 1936: Shewhart’s Statistical Method from the Viewpoint of Quality Control was published. \n\nThese publications and events laid the

## IV. The Role of Kaoru Ishikawa in the Development of TQM
- Introduction to Ishikawa: his background and career
- Ishikawa's contribution to the development of Quality Circles
- The development and importance of the Ishikawa (cause-and-effect) Diagram
- Ishikawa's philosophical contribution to TQM: the 'next process is your customer'


In [201]:
query = "write about:The Role of Kaoru Ishikawa in the Development of TQM - Introduction to Ishikawa: his background and career"
qa.run(query)

'Kaoru Ishikawa was a Japanese professor of engineering, quality management pioneer, and influential contributor to the development of Total Quality Management (TQM). He was born in Nishigahara, Takinogawa Ward, Tokyo City on July 13, 1915. He entered Elementary School attached to Tokyo Higher Normal School at the age of 8, and went on to Junior High School attached to Tokyo Higher Normal School at 13. He graduated from Tokyo High School University at the age of 18 and started his research on coal briquettes. \n\nIn 1948, Ishikawa joined the faculty of the First Faculty of Engineering at Tokyo Imperial University as an associate professor in the Department of Petroleum Refinery Engineering Laboratory. He and Masumasa Imaizumi, then a graduate student, began studying statistical methods at the University of Tokyo.\n\nThroughout his career, Ishikawa made a number of significant contributions to the development of quality management, including: \n\n• Developing Quality Circles and introdu

In [202]:
query = "write about:The Role of Kaoru Ishikawa in the Development of TQM - Ishikawa's contribution to the development of Quality Circles"
qa.run(query)

'Kaoru Ishikawa played a major role in the development of Total Quality Management (TQM) and Quality Circles (QC). In 1965, Ishikawa joined the 2nd QC Overseas Research Team as the head of the delegation and introduced the activities of QC circles at the ASQC Annual Conference. He became a member of the Quality Control Symposium Organizing Committee from 1965 to 1978. He was also a member of the International Cooperation Committee from 1965 to 1978, and a lecturer on the “Production Control Course for Machinery Manufacturers” for participants from developing countries. \n\nIn 1971, Ishikawa became the chair of the ISO/TC 69 Japan Committee and the chair of the organizing committee for the International Symposium on the Air Pollution Prevention Technology. He also became a member of the International Association for Quality (IAQ) in 1971. \n\nIshikawa\'s contribution to the development of Quality Circles was significant. He wrote and edited "Quality Assurance Guidebook" (JUSE Press) whi

In [203]:
query = "write about:The Role of Kaoru Ishikawa in the Development of TQM - The development and importance of the Ishikawa (cause-and-effect) Diagram"
qa.run(query)

'Kaoru Ishikawa was an influential figure in the development of Total Quality Management (TQM). His research and methods have been widely adopted in the modern business world. \n\nIn particular, his work on the Ishikawa (cause-and-effect) diagram has been highly influential. The diagram was first proposed by Ishikawa in 1969 as a way to analyze complex causes and effects in quality management. The Ishikawa diagram is also known as a fishbone diagram, and it is used to identify all of the potential causes of a problem and to analyze the relationship between those causes.\n\nIshikawa’s career began in 1933 when he entered Tokyo Imperial University. He later went on to become an advisor to the Japan Productivity Center and a member of the Japanese Society for Quality Control. He was also a member of the International Cooperation Committee of the American Society for Quality Control (ASQC). \n\nIn 1965, Ishikawa joined the 2nd QC Overseas Research Team as the head of the delegation. This g

In [204]:
query = "write about:The Role of Kaoru Ishikawa in the Development of TQM - Ishikawa's philosophical contribution to TQM: the 'next process is your customer'"
qa.run(query)

'Kaoru Ishikawa\'s contribution to the development of TQM was profound and wide-reaching. He was a key player in the evolution of the concept, helping to shape the principles and practices of TQM in the post-World War II era.\n\nIshikawa\'s efforts can be seen in the various aspects of TQM, including the following:\n\n• Quality Circles: Ishikawa was instrumental in the development of Quality Circles, which are teams of workers who meet regularly to discuss and identify problems and solutions in the workplace. He developed the concept in the late 1950s and it has been widely adopted in Japan and around the world since then.\n\n• Ishikawa (Cause-and-Effect) Diagram: This diagram, also known as the "Fishbone Diagram," is a tool used to help identify and analyze potential causes of problems. It was developed by Ishikawa in the 1960s and has since become a mainstay of quality management practices.\n\n• Next Process Is Your Customer: This philosophy, coined by Ishikawa, emphasizes the import

## V. The Role of the Japanese Union of Scientists and Engineers (JUSE)
- Background of JUSE and its role in the development of quality management in Japan
- Introduction of the Deming Prize by JUSE and its significance
- JUSE's role in promoting TQM and its principles in Japan and globally


In [205]:
query = "write about:The Role of Kaoru Ishikawa in the Development of TQM - Background of JUSE and its role in the development of quality management in Japan"
qa.run(query)

"Kaoru Ishikawa was an influential figure in the development of Total Quality Management (TQM). He was a professor at Gunma University and a part-time lecturer at the Japanese Union of Scientists and Engineers (JUSE). Ishikawa's background and career, his contribution to the development of Quality Circles, the development and importance of the Ishikawa (cause-and-effect) Diagram, and his philosophical contribution to TQM are all important aspects of his impact on the development of TQM.\n\nBackground of JUSE and its Role in the Development of Quality Management in Japan\n\nThe Japanese Union of Scientists and Engineers (JUSE) was founded in 1949 to promote the scientific and technological development of Japan. JUSE played a significant role in the development of quality management in Japan, including hosting seminars and other activities promoting quality control. \n\nIn 1965, JUSE held the Quality Control (QC) and Standardization Seminar, and established the Award of the Excellent Fac

In [206]:
query = "write about:The Role of Kaoru Ishikawa in the Development of TQM - ntroduction of the Deming Prize by JUSE and its significance"
qa.run(query)

"Kaoru Ishikawa's contributions to the development of Total Quality Management (TQM) are extensive and have had a lasting impact. His work, alongside that of the Japanese Union of Scientists and Engineers (JUSE), helped shape the modern quality management landscape. \n\nIshikawa was a Japanese professor and business executive who is widely credited with popularizing the concept of TQM. He developed several influential quality management tools such as the Ishikawa (cause-and-effect) Diagram and Quality Circles. He also introduced the concept of the ‘next process is your customer’, which emphasizes the importance of customer satisfaction in the production process. Additionally, he was involved in the establishment the Deming Prize in 1951, which is an award for excellence in quality control given by JUSE.\n\nIshikawa's work has had a lasting impact on the field of quality management. His tools and methods are still widely used today, and his philosophy of customer satisfaction continues 

In [207]:
query = "write about:The Role of Kaoru Ishikawa in the Development of TQM - JUSE's role in promoting TQM and its principles in Japan and globally"
qa.run(query)

'Kaoru Ishikawa was a critical figure in the development and evolution of Total Quality Management (TQM). He was a renowned professor and quality management consultant who made lasting contributions to the field of quality management. He was a leader in the Japanese Union of Scientists and Engineers (JUSE) and his work was instrumental in promoting TQM in Japan and around the world.\n\nJUSE was a key player in the development of TQM. JUSE was established in 1946 and was charged with developing and promoting TQM in Japan. JUSE held the QC and Standardization Seminar in 1950 and established the Award of the Excellent Factories in Implementation of Industrial Standardization, the Award of the Minister of International Trade and Industry, the Award of Director-General of Agency of Industrial Science and Technology for the Excellent Factory in Implementation of Industrial Standardization and the Award of the Director-General of the Regional Bureau of International Trade and Industry for the

## VI. The Impact of TQM, Ishikawa, and JUSE on Contemporary Business Practices
- Analysis of the role TQM plays in today's business and management practices
- The legacy of Ishikawa's methods and tools in current quality management
- Influence of JUSE's principles in contemporary quality initiatives and awards


In [208]:
query = "write about: The Impact of TQM, Ishikawa, and JUSE on Contemporary Business Practices - Analysis of the role TQM plays in today's business and management practices"
qa.run(query)

'Total Quality Management (TQM) has had a major impact on contemporary business practices. It is a philosophy and set of principles that emphasize the importance of quality in all aspects of business operations. TQM encourages the use of data-driven decision-making, continuous improvement, and customer satisfaction.\n\nKaoru Ishikawa and the Japanese Union of Scientists and Engineers (JUSE) played a pivotal role in the development of TQM. In the years following World War II, Ishikawa and JUSE developed a number of quality improvement tools and methods that are still widely used today. These include the Ishikawa (cause-and-effect) Diagram, Quality Circles, and the Deming Prize.\n\nThe following table outlines the major impacts of TQM, Ishikawa, and JUSE on contemporary business practices:\n\n| Impact | Description |\n| :--- | :--- |\n| Data-driven Decision-Making | TQM encourages businesses to use data to make informed decisions about their operations. This includes gathering data on cu

In [209]:
query = "write about: The Impact of TQM, Ishikawa, and JUSE on Contemporary Business Practices - The legacy of Ishikawa's methods and tools in current quality management"
qa.run(query)

"The legacy of Kaoru Ishikawa's methods and tools in current quality management are evident in the global business landscape. His influence has been wide-reaching, and his contributions have had a lasting impact on business practices. \n\nIshikawa's contributions to quality management are numerous, but the following are particularly noteworthy: \n\n* Quality Circles: Ishikawa developed the concept of Quality Circles in the early 1960s, and the practice has since been widely adopted throughout the world. Quality Circles are a collaborative approach to problem-solving and process improvement, in which teams of employees come together to identify and address quality issues.\n\n* Ishikawa (Cause-and-Effect) Diagram: Ishikawa is credited with the development of the Ishikawa (cause-and-effect) diagram in the late 1960s. This simple graphical tool helps to identify the root causes of problems and to find solutions. It is still widely used in quality management today.\n\n* Next Process is Your

In [210]:
query = "write about: The Impact of TQM, Ishikawa, and JUSE on Contemporary Business Practices - Influence of JUSE's principles in contemporary quality initiatives and awards"
qa.run(query)

'\nThe Impact of TQM, Ishikawa, and JUSE on Contemporary Business Practices\n\nThe legacy of Kaoru Ishikawa, the Japanese Union of Scientists and Engineers (JUSE), and total quality management (TQM) have had a profound impact on contemporary business practices. While Ishikawa and JUSE laid the foundation for modern quality management in the 1950s and 1960s, their influence is still felt in today’s business. Here is an overview of the influence of JUSE’s principles in contemporary quality initiatives and awards:\n\n• Deming Prize: Established by JUSE in 1951, the Deming Prize is the oldest and most prestigious quality management award in the world. It is awarded annually to recognize outstanding achievements in quality control and improvements in business performance.\n\n• ISO/TC 69: JUSE played a major role in establishing the ISO/TC 69 standard in 1980. This standard is used to ensure the quality of products and services in all industries around the world.\n\n• International Standardi

## VII. Conclusion
- Summary of the importance and influence of TQM, Ishikawa, and JUSE in the field of quality management
- Restatement of Thesis
- Future implications of the principles and practices of TQM, Ishikawa, and JUSE in the global business landscape

In [211]:
query = "write about: Summary of the importance and influence of TQM, Ishikawa, and JUSE in the field of quality management"
qa.run(query)

'Kaoru Ishikawa and the Japanese Union of Scientists and Engineers (JUSE) have had a significant impact on the field of Quality Management. \n\nTQM, Ishikawa, and JUSE have contributed to the development of Quality Management principles and practices that are used globally today. \n\nTQM was first developed in Japan in the 1950s as a response to the need for efficient, cost-effective production in the post-World War II era. It was based on the principles of quality circles, management by objectives, and statistical quality control, which were popularized by Kaoru Ishikawa and his work with JUSE.\n\nIshikawa was a key figure in the development of TQM and is credited with introducing the Ishikawa (cause-and-effect) Diagram, which is still used today in quality management initiatives. He also developed the concept of “next process is your customer”, which emphasizes the importance of customer satisfaction in quality management. \n\nJUSE has played an important role in promoting TQM and it

In [212]:
query = "Restatement of Thesis"
qa.run(query)

"\nThe thesis of this paper is that Total Quality Management (TQM), Kaoru Ishikawa, and the Japanese Union of Scientists and Engineers (JUSE) have had a significant impact on contemporary business practices. \n\nTo support this thesis, this paper outlines:\n\n- Definition of Total Quality Management (TQM)\n- Introduction to Kaoru Ishikawa and the Japanese Union of Scientists and Engineers (JUSE)\n- Overview of the principles and practices of TQM\n- The role of TQM in post-World War II Japan's industrial comeback\n- Adoption and evolution of TQM in the United States and around the world\n- Introduction to Ishikawa: his background and career\n- Ishikawa's contribution to the development of Quality Circles\n- The development and importance of the Ishikawa (cause-and-effect) Diagram\n- Ishikawa's philosophical contribution to TQM: the 'next process is your customer'\n- Background of JUSE and its role in the development of quality management in Japan\n- Introduction of the Deming Prize by J

In [213]:
query = "write about: Future implications of the principles and practices of TQM, Ishikawa, and JUSE in the global business landscape"
qa.run(query)

'The future implications of the principles and practices of TQM, Ishikawa, and JUSE in the global business landscape will be far-reaching. As quality management continues to become increasingly important for organizations around the world, the contributions of these three groups will help to shape the future of the industry. \n\nThe principles of TQM, developed by Kaoru Ishikawa and the Japanese Union of Scientists and Engineers (JUSE), have become the foundation for modern quality management systems. The focus on process improvement, customer satisfaction, and continuous learning has become a cornerstone of the industry. By continuing to emphasize these principles, organizations will be able to create more efficient processes and deliver higher quality products and services. \n\nIshikawa’s methods and tools, such as the Ishikawa (cause-and-effect) Diagram and Quality Circles, are also providing organizations with the tools they need to improve their processes and increase customer sat